In [1]:
import os
original_dir = os.getcwd()
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))
exec(open('setup/default.py').read())

In [17]:
menu_df = pd.read_csv(os.getcwd()+'/data/preproc/main/menu.csv').drop(columns=['type','writer','body','time','hour'])
sold_out_df = pd.read_csv(os.getcwd()+'/data/preproc/main/soldout.csv').drop(columns=['type','writer','time','body'])
sold_out_df .rename(columns={'hour': 'soldout_time'}, inplace=True)


In [18]:
sold_out_df['soldout'] = sold_out_df['soldout'].apply(ast.literal_eval) # 리스트 타입으로 바꿔주기 
sold_out_df  = sold_out_df.explode('soldout')
sold_out_df['soldout']  = sold_out_df['soldout'].astype(int)

In [22]:
import pandas as pd

# 예시 데이터프레임 생성
df1 = pd.DataFrame({
    'key1': [[1, 2], [1], [2]],
    'value1': ['a', 'b', 'c']
})

df2 = pd.DataFrame({
    'key2': [1, 2],
    'value2': ['x', 'y']
})

# df1과 df2를 key1과 key2를 기준으로 left 조인
# 각 key1 리스트에 대해 key2 값이 포함된 모든 경우를 조인

# 빈 리스트를 초기화할 리스트 선언
joined_data = []

# df1을 순환하면서, key1 리스트 안에 있는 모든 key2와 key2를 매핑
for idx, row in df1.iterrows():
    for key2 in row['key1']:
        for key2 in df2['key2']:
            joined_data.append([row['key1'], row['value1'], key2, df2.loc[df2['key2'] == key2, 'value2'].values[0]])

# 결과 데이터프레임 생성
result_df = pd.DataFrame(joined_data, columns=['key1', 'value1', 'key2', 'value2'])

print(result_df)

     key1 value1  key2 value2
0  [1, 2]      a     1      x
1  [1, 2]      a     2      y
2  [1, 2]      a     1      x
3  [1, 2]      a     2      y
4     [1]      b     1      x
5     [1]      b     2      y
6     [2]      c     1      x
7     [2]      c     2      y
